# 从 ConversationBufferMemory 或 ConversationStringBufferMemory 迁移

[ConversationBufferMemory](https://python.langchain.com/api_reference/langchain/memory/langchain.memory.buffer.ConversationBufferMemory.html) 和 [ConversationStringBufferMemory](https://python.langchain.com/api_reference/langchain/memory/langchain.memory.buffer.ConversationStringBufferMemory.html) 用于在没有任何额外处理的情况下跟踪人类和 AI 助手之间的对话。

:::note
`ConversationStringBufferMemory` 等同于 `ConversationBufferMemory`，但它是针对非聊天模型的 LLM 设计的。
:::

使用现有现代方法处理对话历史的方法有：

1. 使用 [LangGraph persistence](https://langchain-ai.github.io/langgraph/how-tos/persistence/) 并对消息历史进行适当处理
2. 使用 LCEL 结合 [RunnableWithMessageHistory](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html#) 并对消息历史进行适当处理。

对于大多数用户来说，[LangGraph persistence](https://langchain-ai.github.io/langgraph/how-tos/persistence/) 比等效的 LCEL 更易于使用和配置，尤其是在更复杂的用例中。

## 设置

In [1]:
%%capture --no-stderr
%pip install --upgrade --quiet langchain-openai langchain

In [2]:
import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass()

## 与 LLMChain / ConversationChain 的用法

本节将展示如何迁移与 `LLMChain` 或 `ConversationChain` 结合使用的 `ConversationBufferMemory` 或 `ConversationStringBufferMemory`。

### 旧用法

下面是 `ConversationBufferMemory` 与 `LLMChain` 或等效的 `ConversationChain` 一起使用的示例。

<details open>

In [4]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate(
    [
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

# highlight-start
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
# highlight-end

legacy_chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=prompt,
    # highlight-next-line
    memory=memory,
)

legacy_result = legacy_chain.invoke({"text": "my name is bob"})
print(legacy_result)

legacy_result = legacy_chain.invoke({"text": "what was my name"})

{'text': 'Hello Bob! How can I assist you today?', 'chat_history': [HumanMessage(content='my name is bob', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={}, response_metadata={})]}


In [5]:
legacy_result["text"]

'Your name is Bob. How can I assist you today, Bob?'

:::note
请注意，不支持在单个内存对象中分离对话线程
:::

</details>

### LangGraph

下面的示例展示了如何使用 LangGraph 实现带有 `ConversationBufferMemory` 的 `ConversationChain` 或 `LLMChain`。

本示例假设您已经对 `LangGraph` 有一定的了解。如果您还不了解，请参阅 [LangGraph Quickstart Guide](https://langchain-ai.github.io/langgraph/tutorials/introduction/) 获取更多详细信息。

`LangGraph` 提供了许多额外的功能（例如，时间旅行和中断），并且非常适合其他更复杂（也更实际）的架构。

<details open>

In [6]:
import uuid

from IPython.display import Image, display
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define a chat model
model = ChatOpenAI()


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    # We return a list, because this will get added to the existing list
    return {"messages": response}


# Define the two nodes we will cycle between
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)


# Adding memory is straight forward in langgraph!
# highlight-next-line
memory = MemorySaver()

app = workflow.compile(
    # highlight-next-line
    checkpointer=memory
)


# The thread id is a unique key that identifies
# this particular conversation.
# We'll just generate a random uuid here.
# This enables a single application to manage conversations among multiple users.
thread_id = uuid.uuid4()
# highlight-next-line
config = {"configurable": {"thread_id": thread_id}}


input_message = HumanMessage(content="hi! I'm bob")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

# Here, let's confirm that the AI remembers our name!
input_message = HumanMessage(content="what was my name?")
for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

hi! I'm bob
================================== Ai Message ==================================

Hello Bob! How can I assist you today?
================================ Human Message =================================

what was my name?
================================== Ai Message ==================================

Your name is Bob. How can I help you today, Bob?


</details>

### LCEL RunnableWithMessageHistory

或者，如果你有一个简单的链，可以将链的聊天模型包装在 [RunnableWithMessageHistory](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html) 中。

有关更多信息，请参阅以下[迁移指南](/docs/versions/migrating_chains/conversation_chain/)。


## 用法示例（预构建代理）

此示例展示了如何将 Agent Executor 与使用 [create_tool_calling_agent](https://python.langchain.com/api_reference/langchain/agents/langchain.agents.tool_calling_agent.base.create_tool_calling_agent.html) 函数构建的预构建代理一起使用。

如果你正在使用一个[旧的 LangChain 预构建代理](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/)，你应该能够用新的 [langgraph 预构建代理](https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/) 替换该代码。新的代理利用了聊天模型的原生工具调用功能，并且开箱即用效果可能更好。

### 旧版用法

<details open>

In [7]:
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.memory import ConversationBufferMemory
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0)


@tool
def get_user_age(name: str) -> str:
    """Use this tool to find the user's age."""
    # This is a placeholder for the actual implementation
    if "bob" in name.lower():
        return "42 years old"
    return "41 years old"


tools = [get_user_age]

prompt = ChatPromptTemplate.from_messages(
    [
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# Construct the Tools agent
agent = create_tool_calling_agent(model, tools, prompt)
# Instantiate memory
# highlight-start
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
# highlight-end

# Create an agent
agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    # highlight-next-line
    memory=memory,  # Pass the memory to the executor
)

# Verify that the agent can use tools
print(agent_executor.invoke({"input": "hi! my name is bob what is my age?"}))
print()
# Verify that the agent has access to conversation history.
# The agent should be able to answer that the user's name is bob.
print(agent_executor.invoke({"input": "do you remember my name?"}))

{'input': 'hi! my name is bob what is my age?', 'chat_history': [HumanMessage(content='hi! my name is bob what is my age?', additional_kwargs={}, response_metadata={}), AIMessage(content='Bob, you are 42 years old.', additional_kwargs={}, response_metadata={})], 'output': 'Bob, you are 42 years old.'}

{'input': 'do you remember my name?', 'chat_history': [HumanMessage(content='hi! my name is bob what is my age?', additional_kwargs={}, response_metadata={}), AIMessage(content='Bob, you are 42 years old.', additional_kwargs={}, response_metadata={}), HumanMessage(content='do you remember my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Yes, your name is Bob.', additional_kwargs={}, response_metadata={})], 'output': 'Yes, your name is Bob.'}


</details>

### LangGraph

你可以遵循 LangChain 的标准教程来[构建一个代理](/docs/tutorials/agents/)，其中有关于此工作原理的深入解释。

此处展示这个示例是为了让用户更容易地比较遗留实现与对应的 LangGraph 实现。

此示例展示了如何在 LangGraph 的[预构建 React 代理](https://langchain-ai.github.io/langgraph/reference/prebuilt/#langgraph.prebuilt.chat_agent_executor.create_react_agent) 中添加内存。

更多详情，请参阅 LangGraph 中关于[如何为预构建 ReAct 代理添加内存](https://langchain-ai.github.io/langgraph/how-tos/create-react-agent-memory/)的指南。

<details open>

In [8]:
import uuid

from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent


@tool
def get_user_age(name: str) -> str:
    """Use this tool to find the user's age."""
    # This is a placeholder for the actual implementation
    if "bob" in name.lower():
        return "42 years old"
    return "41 years old"


# highlight-next-line
memory = MemorySaver()
model = ChatOpenAI()
app = create_react_agent(
    model,
    tools=[get_user_age],
    # highlight-next-line
    checkpointer=memory,
)

# highlight-start
# The thread id is a unique key that identifies
# this particular conversation.
# We'll just generate a random uuid here.
# This enables a single application to manage conversations among multiple users.
thread_id = uuid.uuid4()
config = {"configurable": {"thread_id": thread_id}}
# highlight-end

# Tell the AI that our name is Bob, and ask it to use a tool to confirm
# that it's capable of working like an agent.
input_message = HumanMessage(content="hi! I'm bob. What is my age?")

for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

# Confirm that the chat bot has access to previous conversation
# and can respond to the user saying that the user's name is Bob.
input_message = HumanMessage(content="do you remember my name?")

for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

hi! I'm bob. What is my age?
================================== Ai Message ==================================
Tool Calls:
  get_user_age (call_oEDwEbIDNdokwqhAV6Azn47c)
 Call ID: call_oEDwEbIDNdokwqhAV6Azn47c
  Args:
    name: bob
================================= Tool Message =================================
Name: get_user_age

42 years old
================================== Ai Message ==================================

Bob, you are 42 years old! If you need any more assistance or information, feel free to ask.
================================ Human Message =================================

do you remember my name?
================================== Ai Message ==================================

Yes, your name is Bob. If you have any other questions or need assistance, feel free to ask!


如果我们使用不同的 thread ID，它将开启一个新的对话，机器人将不知道我们的名字！

In [9]:
config = {"configurable": {"thread_id": "123456789"}}

input_message = HumanMessage(content="hi! do you remember my name?")

for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

hi! do you remember my name?
================================== Ai Message ==================================

Hello! Yes, I remember your name. It's great to see you again! How can I assist you today?


</details>

## 后续步骤

使用 LangGraph 探索持久化：

* [LangGraph 快速入门教程](https://langchain-ai.github.io/langgraph/tutorials/introduction/)
* [如何为图添加持久化（“内存”）](https://langchain-ai.github.io/langgraph/how-tos/persistence/)
* [如何管理对话历史](https://langchain-ai.github.io/langgraph/how-tos/memory/manage-conversation-history/)
* [如何添加对话历史摘要](https://langchain-ai.github.io/langgraph/how-tos/memory/add-summary-conversation-history/)

通过简单的 LCEL 添加持久化（对于更复杂的用例，建议使用 langgraph）：

* [如何添加消息历史](/docs/how_to/message_history/)

处理消息历史：

* [如何修剪消息](/docs/how_to/trim_messages)
* [如何过滤消息](/docs/how_to/filter_messages/)
* [如何合并消息运行](/docs/how_to/merge_message_runs/)